# The matrix sum problem

## Dense layers

The matrix sum problem (https://projecteuler.net/problem=345) was fun, and I learnt a lot via looking at MrDrake's DP solution (https://projecteuler.net/thread=345#36977), which was far better than mine, both time and space-wise. Because it runs in under a tenth of a second on my machine,
I can use it to get the labels of a dataset I can easily create with Numpy, and then train a neural network on this regression task.
Let's see how a Tensorflow model does on this problem!



In [1]:
#from matrix_sum import dp_solution

import numpy as np

from tensorflow.keras import Sequential, layers, models
import tensorflow as tf

from sklearn.model_selection import train_test_split

2023-04-25 16:55:20.058241: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-25 16:55:20.058395: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
GRID_SIZE = (5, 5) 

In [3]:
def dp_solution(matrix: np.ndarray):
    """Submitted by user MrDrake from Australia on Project Euler,
    on Sept 4th 2011. This solution leverages dynamic programming
    and is way, way faster than mine (180 times, or about 2 orders of magnitude!)
    while being also more compact, using less dependencies and probably less memory!
    Hats off to you, MrDrake!"""
    MINUS_INF = -float("inf")
    n = len(matrix)
    dp = {0: 0}
    # key is a bitmask representing the set of columns already visited,
    # value is the max sum of the path
    for row in range(n):
        z = {}
        for column in range(n):
            x = 1 << column
            # set the bit of the current column to 1, all other remain 0
            for d in dp:
                if x & d:
                    # if in this mask, column `column` is visited, skip
                    continue
                y = matrix[row][column] + dp[d]
                # path sum = current cell weight + previous path sum
                if z.get(x | d, MINUS_INF) < y:
                    z[x | d] = y  # update the max path sum
        dp = z
    return dp[(1 << n) - 1]  # (1 << n) - 1 is the mask with all bits set to 1,
    # meaning we want the max path sum for all columns

In [4]:
# Création de l'architecture du modèle

model = Sequential(name = 'simple_model')
model.add(layers.Flatten(input_shape=GRID_SIZE))
model.add(layers.Dense(50, activation='relu', name = 'hidden_layer_1'))
model.add(layers.Dense(50, activation='relu', name = 'hidden_layer_2'))
model.add(layers.Dense(50, activation='relu', name = 'hidden_layer_3'))
model.add(layers.Dense(50, activation='relu', name = 'hidden_layer_4'))
model.add(layers.Dense(25, activation='relu', name = 'hidden_layer_5'))
model.add(layers.Dense(5, activation='relu', name = 'hidden_layer_6'))
model.add(layers.Dense(1, activation='linear', name='output_neuron')) # la sortie


# On 'compile' le modèle: essentiellement, on définit la loss, la metric, et l'optimizer

model.compile(loss='mse', optimizer='adam', metrics=['mae'])
#model.summary()

2023-04-25 16:55:36.151011: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-25 16:55:36.158102: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-25 16:55:36.158190: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-04-25 16:55:36.158247: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-04-25 16:55:36.158358: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [13]:
NB_EXAMPLES = 100_000
X = np.random.randint(0, 1_000, (NB_EXAMPLES, *GRID_SIZE))
y = np.array([dp_solution(grid) for grid in X])
print(X.shape, y.shape)

(100000, 5, 5) (100000,)


In [14]:
history = model.fit(X, y, epochs=15, batch_size = 32, verbose=1) ;

Epoch 1/15
3125/3125 [==============================] - 11s 3ms/step - loss: 31593.2188 - mae: 141.3444
Epoch 2/15
3125/3125 [==============================] - 10s 3ms/step - loss: 27824.9492 - mae: 132.7811
Epoch 3/15
3125/3125 [==============================] - 10s 3ms/step - loss: 24127.1777 - mae: 123.6798
Epoch 4/15
3125/3125 [==============================] - 10s 3ms/step - loss: 21371.9023 - mae: 116.3808
Epoch 5/15
3125/3125 [==============================] - 11s 3ms/step - loss: 18678.0742 - mae: 108.8631
Epoch 6/15
3125/3125 [==============================] - 12s 4ms/step - loss: 16965.7930 - mae: 103.8850
Epoch 7/15
3125/3125 [==============================] - 10s 3ms/step - loss: 15454.7715 - mae: 99.1124
Epoch 8/15
3125/3125 [==============================] - 10s 3ms/step - loss: 14433.9912 - mae: 95.9085
Epoch 9/15
3125/3125 [==============================] - 10s 3ms/step - loss: 13563.1650 - mae: 92.9004
Epoch 10/15
3125/3125 [==============================] - 10s 3ms/st

In [35]:
test_grid = np.random.randint(0, 1_000, GRID_SIZE)
test_grid

array([[ 28, 413, 955, 695, 577],
       [291, 603, 231, 834, 191],
       [875, 474, 996, 317, 639],
       [878, 400, 145, 400, 256],
       [655, 287, 211, 115, 787]])

Ground truth

In [36]:
dp_solution(test_grid)

3959

Prediction

In [37]:
model.predict(test_grid.reshape(1, *GRID_SIZE))

array([[4077.5342]], dtype=float32)

model does badly, it'd likely be better to use a convnet instead, due to the grid shape of the inputs

## Convnet

In [45]:
cnn = Sequential()
cnn.add(layers.Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(5,5,1)))
cnn.add(layers.Conv2D(4, kernel_size=(3), activation='relu')) # kernel_size = 3 <==> (3, 3)
cnn.add(layers.Flatten())
cnn.add(layers.Dense(1, activation='linear')) 

cnn.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 3, 3, 6)           60        
                                                                 
 conv2d_8 (Conv2D)           (None, 1, 1, 4)           220       
                                                                 
 flatten_2 (Flatten)         (None, 4)                 0         
                                                                 
 dense_1 (Dense)             (None, 1)                 5         
                                                                 
Total params: 285
Trainable params: 285
Non-trainable params: 0
_________________________________________________________________


In [46]:
cnn.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [47]:
history = cnn.fit(X, y, epochs=15, batch_size = 32, verbose=1) ;

Epoch 1/15
3125/3125 [==============================] - 8s 2ms/step - loss: 282511.9062 - mae: 290.0032
Epoch 2/15
3125/3125 [==============================] - 7s 2ms/step - loss: 51112.2109 - mae: 179.8060
Epoch 3/15
3125/3125 [==============================] - 7s 2ms/step - loss: 47082.7734 - mae: 172.5562
Epoch 4/15
3125/3125 [==============================] - 7s 2ms/step - loss: 44613.5664 - mae: 167.9409
Epoch 5/15
3125/3125 [==============================] - 7s 2ms/step - loss: 42795.7812 - mae: 164.3707
Epoch 6/15
3125/3125 [==============================] - 7s 2ms/step - loss: 41443.9883 - mae: 161.9072
Epoch 7/15
3125/3125 [==============================] - 7s 2ms/step - loss: 40469.9258 - mae: 159.9222
Epoch 8/15
3125/3125 [==============================] - 7s 2ms/step - loss: 39760.0352 - mae: 158.6573
Epoch 9/15
3125/3125 [==============================] - 7s 2ms/step - loss: 39172.2031 - mae: 157.4951
Epoch 10/15
3125/3125 [==============================] - 7s 2ms/step - l

KeyboardInterrupt: 